In [ ]:
#Folium Test Program
# To install folium to your computer run the following:
# conda install --channel https://conda.anaconda.org/conda-forge folium
# We will also want to install shapely to do a bit of geocoding, so to install this run:
# conda install shapely


In [ ]:
import folium
import pandas as pd
import numpy as np
from datetime import datetime
import urllib
import json

In [ ]:
map_osm = folium.Map(location=[37.79086, -122.40147], zoom_start=14)
folium.Marker([37.79086, -122.40147], popup='General Assembly').add_to(map_osm)
map_osm

In [ ]:
def read_geo_url(url):
    in_file = urllib.urlopen(url)
    return json.load(in_file)

In [ ]:
state_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/us-states.json'
state_geo = read_geo_url(state_url)
state_unemployment = (
    r'https://raw.githubusercontent.com/python-visualization/folium/master/examples/US_Unemployment_Oct2012.csv')

state_data = pd.read_csv(state_unemployment)

state_data.head()

# To view json go to the following link:
# https://raw.githubusercontent.com/python-visualization/folium/master/examples/us-states.json

In [ ]:
#Let Folium determine the scale and view Choropleth
states = folium.Map(location=[40, -102], zoom_start=4)
states.choropleth(geo_str=state_geo, data=state_data,
                columns=['State', 'Unemployment'],
                key_on='feature.id',
                fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
                legend_name='Unemployment Rate (%)')
states
# To load a choropleth map from your hard drive use geo_path (i.e.)
# states.choropleth(geo_path='states.geojson', data=state_data,
#                columns=['State', 'Unemployment'], 
#                key_on='feature.id',
#                fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
#                legend_name='Unemployment Rate (%)')

Lets start with a more local version 

A topic near and dear to my heart, human feces in San Francisco

View this map published by the amazing William Mees. https://willyem.carto.com/viz/156b1e0c-5b45-11e5-9351-0e018d66dc29/public_map

http://www.citylab.com/housing/2015/10/mapping-san-franciscos-sidewalk-pooping-problem/409561/

My plan for this exercise is to import 
in the GeoJson file for human feces over the last few years 
in San Francisco, as a way to visualize how the data is spread out over the years.

I will also be using a San Francisco geojson map of neighborhoods, and combining this with a dataset of excrement sightings to visualize it as an aggregate of data per neighborhood.

I will first just show a simple map of the points shown in the data set that William gathered. 

In [ ]:
poop_url = r"https://raw.githubusercontent.com/joshuacano/DS-SF-24/master/folium_lecture/2010_poop.json"
poop_data = read_geo_url(poop_url)

In [ ]:
# Let's look at the first poop instance!
poop_data['features'][0]

In [ ]:
print len(poop_data['features'])

# We should only use 500 observations as folium will stall ipython with any more points
poop_data['features'] = poop_data['features'][:500] 
poop_map = folium.Map(location=[37.79086, -122.40147], zoom_start=12)
poop_map.choropleth(geo_str=poop_data)
poop_map

In [ ]:
#Lets overlay that with some neighborhood shapes.
# I took this data from code america. Thanks Code America!
# https://github.com/codeforamerica/click_that_hood/blob/master/public/data/san-francisco.geojson

sf_path='https://raw.githubusercontent.com/joshuacano/DS-SF-24/master/folium_lecture/san-francisco.geojson'
hood_json = read_geo_url(sf_path)

#Add style attributes
my_style = {
    "color": "#ff7800",
    "weight": 2,
    "opacity": 0.65
}
folium.GeoJson(hood_json,
               style_function=lambda x: my_style, overlay=True).add_to(poop_map)

poop_map

Okay, so this looks a little bit funky. But its got some good information on it, 
so lets try and aggregate all this data together using pandas!

First, lets read our data into a dataframe and get the details we need from it

In [ ]:
# Seems like we will want the point, the date it was opened, the casid, 
# the neighborhood, and the supervisor_district, that could be interesting!
def get_details(item):
    details = {}
    details['point'] = item['geometry']['coordinates']
    details['date_opened'] = item['properties']['opened']
    details['caseid'] = item['properties']['caseid']
    details['neighborhood'] = item['properties']['neighborhood']
    details['supervisor_district'] = item['properties']['supervisor_district']
    return details

poop_url = r"https://raw.githubusercontent.com/joshuacano/DS-SF-24/master/folium_lecture/2010_poop.json"
poop_data = read_geo_url(poop_url)

rs = []
for incident in poop_data['features']:
    rs.append(get_details(incident))
poop_frame = pd.DataFrame(rs)
print len(poop_frame)
poop_frame.head()

In [ ]:
poop_frame.groupby(['neighborhood']).count()

In [ ]:
# K so it seems like we have a much bigger amount of neighborhoods in the incident data,
# than in our neighborhood layer. 
# lets try and match each incident to the existing neighborhood layer we got.
# For that we will use the shapely library we just downloaded.
import shapely.geometry
point = shapely.geometry.Point (poop_frame.point[0])

In [ ]:
print datetime.now()
hood_shapes = []

for feature in hood_json["features"]:
    hood_shapes.append({
            "shape" : shapely.geometry.shape(feature["geometry"]),
             "name" : feature['properties']['name']})
    

poop_frame['shape_neighborhood'] = None
for hood_shape in hood_shapes:
    for row in poop_frame.loc[poop_frame['shape_neighborhood'].isnull(),:].itertuples():
        point = shapely.geometry.Point(row.point)
        # This will check to see if the incident was in the neighborhood
        if hood_shape['shape'].contains(point):
            poop_frame.loc[row[0], 'shape_neighborhood'] = hood_shape['name']
print datetime.now()

In [ ]:
# Ok so now that we know how to match an incident with our existing neighborhood geojson file,
# lets make a new dataframe that has all the neighborhoods with their frequency per neighborhood 
freq_frame = poop_frame.groupby(['shape_neighborhood']).count()
freq_frame['shape_neighborhood'] = freq_frame.index

In [ ]:
# In case we have some points not in neighborhoods lets setup a default case
def fill_with_default(hood_shapes, frame):
    default_frame = pd.DataFrame(hood_shapes)
    default_frame['caseid'] = 0 
    default_frame['shape_neighborhood'] = default_frame['name']
    del default_frame['shape']
    default_frame.set_index(['name'], inplace=True)
    default_frame.update(frame)
    default_frame.caseid = default_frame.caseid.astype(int)
    return default_frame

freq_frame = fill_with_default(hood_shapes, freq_frame)   

In [ ]:
# Now lets associate it with our original neighborhood map.
poop_map = folium.Map(location=[37.79086, -122.40147], zoom_start=12)
poop_map.choropleth(geo_str=hood_json,
                    data=freq_frame,
                columns=['shape_neighborhood', 'caseid'],
                key_on='feature.properties.name',
                fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
                legend_name='Poop Frequency')
poop_map

In [ ]:
# We can see the automated legend is thrown off by the outliers, lets send in our own legend
poop_map = folium.Map(location=[37.79086, -122.40147], zoom_start=12)
poop_map.choropleth(geo_str=hood_json,
                    data=freq_frame,
                columns=['shape_neighborhood', 'caseid'],
                threshold_scale=[5,20,40,60,80, 400],
                key_on='feature.properties.name',
                fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
                legend_name='Poop Frequency')

poop_map

In [ ]:
def get_daterange(start_date, end_date, frame):
    """In case anyone wants to cut the data by start and end_date"""
    df = frame.groupby(['shape_neighborhood']).count()
    df.caseid = 0
    date_frame = frame[(frame.date_opened >= start_date) &
                       (frame.date_opened < end_date)]
    date_frame = date_frame.groupby(['shape_neighborhood']).count()['caseid']
    df.update(date_frame)
    df.caseid = df.caseid.astype(int)
    df['shape_neighborhood'] = df.index
    return df

def get_threshold_scale(frame, column):
    """In case you want to have a customized Threshold scale for the dataFrame
    
    We will use a larger than normal range for the first 4 scales.
    In order to get some more separation for the tiles"""
    rs = []
    for i in np.linspace(0.2, 0.95, 6, endpoint=True):
        rs.append(frame[frame[column] >0][column].quantile(i))
    return rs

poop_map = folium.Map(location=[37.79086, -122.40147], zoom_start=12)
poop_map.choropleth(geo_str=hood_json,
                    data=freq_frame,
                columns=['shape_neighborhood', 'caseid'],
                threshold_scale=(get_threshold_scale(freq_frame, 'caseid')),
                key_on='feature.properties.name',
                fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
                legend_name='Poop Frequency')

poop_map